In [52]:
import json
import os
from pyspark.ml import Pipeline
from sparknlp.base import *
from sparknlp.annotator import *
import sparknlp
import pandas as pd
import openpyxl
import numpy as np
spark = sparknlp.start()

def get_ann_pipeline ():
    
    document_assembler = DocumentAssembler() \
        .setInputCol("text")\
        .setOutputCol('document')

    sentence = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentence')\
        .setCustomBounds(['\n'])

    tokenizer = Tokenizer() \
        .setInputCols(["sentence"]) \
        .setOutputCol("token")

    pos = PerceptronModel.pretrained() \
              .setInputCols(["sentence", "token"]) \
              .setOutputCol("pos")
    
    embeddings = WordEmbeddingsModel.pretrained()\
          .setInputCols(["sentence", "token"])\
          .setOutputCol("embeddings")

    ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
       .setInputCols(["sentence", "token", "embeddings"])\
       .setOutputCol("ner")

    ner_converter = NerConverter()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk")

    ner_pipeline = Pipeline(
        stages = [
            document_assembler,
            sentence,
            tokenizer,
            pos,
            embeddings,
            ner_model,
            ner_converter
        ]
    )

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    ner_pipelineFit = ner_pipeline.fit(empty_data)

    ner_lp_pipeline = LightPipeline(ner_pipelineFit)
    print ("Spark NLP NER lightpipeline is created")

    return ner_lp_pipeline

In [53]:
conll_pipeline = get_ann_pipeline ()

pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
Spark NLP NER lightpipeline is created


2023-01-17 10:23:06.242736: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [54]:
parsed = conll_pipeline.annotate ("Where am I")
parsed

sentances  = ["i want to make coffee"]
sentances.append("where am i")
sentances.append("what can i do here")
sentances.append("prepare meal")
sentances.append("use refrigerator")
sentances.append("use fan")
sentances.append("use oven")
sentances.append("use stove")
sentances.append("i would like to wash sheets")
sentances.append("i would like to watch television")
sentances.append("wash sheets")
sentances.append("wash clothes")
sentances.append("cook dinner")
sentances.append("i would like to cook dinner")
sentances.append("where is the coffee")
sentances.append("rewind video")
sentances.append("pause video")
sentances.append("skip forward")
sentances.append("negative")
sentances.append("positive")
sentances.append("yes")
sentances.append("no")
sentances.append("hello yes")
sentances.append("hello no")
sentances.append("shutdown app")
sentances.append("shut")
sentances.append("hello")
sentances.append("play video")
sentances.append("hello shut down app")
sentances.append("hello shut")
sentances.append("hello play video")
sentances.append("hello negative")
sentances.append("hello positive")
sentances.append("correct")
sentances.append("incorrect")
sentances.append("hello correct")
sentances.append("hello incorrect")
sentances.append("i want to make coffee")
sentances.append("where am i")


df = pd.read_excel("./Keyword Map NLP.xlsx")
Loc = df.loc[df['Loc']==1]
#print(Loc)
workbook = openpyxl.load_workbook('./Keyword Map NLP.xlsx')
sh = workbook.active
                 
get= []
print("start")
for i in range(1,sh.max_column+1):
    #print(i)
    search = sh.cell(row=1,column=i).value
    #print(search)
    if(search == 'Loc'):
        for j in range(2,sh.max_row+1):
            one = sh.cell(row=j,column=i).value
            if(one == 1):
                #print(sh.cell(row=j,column=1).value)
                get.append(sh.cell(row=j,column=1).value)
        break
                 
for w in get:
    filer = (f"i want to go to the {w}")
    sentances.append(filer)

for w in get:
    filer = (f"navigate to {w}")
    sentances.append(filer)
                 
for i in range(1,sh.max_row+1):
    keyword = sh.cell(row=i,column=1).value
    sentances.append(keyword)                          


with open('check.train', 'w') as file:
      

    file.writelines("% s\n" % data for data in sentances)

start


In [55]:
store = ["-DOCSTART- -X- -X- O\n"]

conll_lines=''

for l in sentances:
    parsed = conll_pipeline.annotate (l)
    for token, pos, ner in zip(parsed['token'],parsed['pos'],parsed['ner']):

        conll_lines += "{} {} {} {}\n".format(token, pos, pos, ner)
        
    conll_lines+="\n"
    store.append(conll_lines)


with open('eng_custom_bert.train', 'w') as file:
      

    file.writelines("% s\n" % data for data in store)


In [56]:
sentances  = ["hello i want to make coffee"]
for w in get:
    filer = (f"hello i want to go to the {w}")
    sentances.append(filer)
for w in get:
    filer = (f"hello navigate to {w}")
    sentances.append(filer)

#filer = (f"hello i want to make coffee")
#sentances.append(filer)
filer = (f"hello where am i")
sentances.append(filer)
filer = (f"hello what can i do here")
sentances.append(filer)
filer = (f"hello prepare meal")
sentances.append(filer)
filer = (f"hello use refrigerator")
sentances.append(filer)
filer = (f"hello start camera")
sentances.append(filer)

filer = (f"hello correct")
sentances.append(filer)
filer = (f"hello incorrect")
sentances.append(filer)
filer = (f"hello play video")
sentances.append(filer)
filer = (f"hello negative")
sentances.append(filer)
filer = (f"hello positive")
sentances.append(filer)


filer = (f"hello yes")
sentances.append(filer)
filer = (f"hello no")
sentances.append(filer)
filer = (f"hello location")
sentances.append(filer)
filer = (f"hello location.")
sentances.append(filer)
filer = (f"hello activity")
sentances.append(filer)
with open('check.testa', 'w') as file:
      

    file.writelines("% s\n" % data for data in sentances)

In [57]:
store = ["-DOCSTART- -X- -X- O\n"]

conll_lines=''

for l in sentances:
    parsed = conll_pipeline.annotate (l)
    for token, pos, ner in zip(parsed['token'],parsed['pos'],parsed['ner']):

        conll_lines += "{} {} {} {}\n".format(token, pos, pos, ner)
        
    conll_lines+="\n"
    store.append(conll_lines)


with open('eng_custom_bert.testa', 'w') as file:
      

    file.writelines("% s\n" % data for data in store)

In [58]:
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *
from sparknlp.pretrained import PretrainedPipeline

from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, './eng_custom_bert.train')
training_data.show()
testing_data= CoNLL().readDataset(spark, './eng_custom_bert.testa')

23/01/17 10:23:19 WARN TaskSetManager: Stage 45 contains a task of very large size (2331 KiB). The maximum recommended task size is 1000 KiB.
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|i want to make co...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 0, i,...|[{pos, 0, 0, NNP,...|[{named_entity, 0...|
|i want to make co...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 0, i,...|[{pos, 0, 0, NNP,...|[{named_entity, 0...|
|          where am i|[{document, 0, 9,...|[{document, 0, 9,...|[{token, 0, 4, wh...|[{pos, 0, 4, WRB,...|[{named_entity, 0...|
|i want to make co...|[{document, 0, 20...|[{document, 0, 20...|[{token, 0, 0, i,...|[{pos

In [59]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)



nerTagger = NerDLApproach()\
    .setInputCols(["sentence", "token", "bert"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(100)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    .setTestDataset("test_withEmbeds.parquet")


test_data = CoNLL().readDataset(spark, './eng.testa')
test_data = bert.transform(test_data)

import os
 
# Directory name
directory = "test_withEmbeds.parquet"
 
# Parent Directory
parent = "/home/sjhjrok/Documents/NLP/"
 
# Path
path = os.path.join(parent, directory)
 
# Remove the Directory
# "GeeksforGeeks"
try:
    os.rmdir(path)
    print("Directory '% s' has been removed successfully" % directory)
except OSError as error:
    print(error)
    print("Directory '% s' can not be removed" % directory)


test_data.write.parquet("test_withEmbeds.parquet")


ner_pipeline = Pipeline(stages = [bert, nerTagger])
ner_model = ner_pipeline.fit(training_data)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]
[Errno 2] No such file or directory: '/home/sjhjrok/Documents/NLP/test_withEmbeds.parquet'
Directory 'test_withEmbeds.parquet' can not be removed
23/01/17 10:23:22 WARN TaskSetManager: Stage 46 contains a task of very large size (1128 KiB). The maximum recommended task size is 1000 KiB.


23/01/17 10:24:29 WARN TaskSetManager: Stage 48 contains a task of very large size (2331 KiB). The maximum recommended task size is 1000 KiB.


23/01/17 10:26:50 WARN TaskSetManager: Stage 49 contains a task of very large size (2331 KiB). The maximum recommended task size is 1000 KiB.


2023-01-17 10:29:13.882791: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

Training started - total epochs: 300 - lr: 0.003 - batch size: 32 - labels: 9 - chars: 22 - training examples: 22064
Epoch 1/300 started, lr: 0.003, dataset size: 22064
Epoch 1/300 - 14.20s - loss: 213.25464 - batches: 691
Quality on validation dataset (20.0%), validation examples = 4412
time to finish evaluation: 1.15s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 1218	 0	 6	 1.0	 0.99509805	 0.997543
Loc	 1866	 0	 3	 1.0	 0.99839485	 0.99919677
Act	 3510	 10	 0	 0.99715906	 1.0	 0.99857754
Neg	 280	 0	 0	 1.0	 1.0	 1.0
0	 5431	 4	 0	 0.999264	 1.0	 0.9996319
Obj	 1562	 0	 5	 1.0	 0.9968092	 0.998402
Pos	 295	 0	 0	 1.0	 1.0	 1.0
Trigger	 425	 0	 0	 1.0	 1.0	 1.0
tp: 14587 fp: 14 fn: 14 labels: 8
Macro-average	 prec: 0.99955285, rec: 0.99878776, f1: 0.9991701
Micro-average	 prec: 0.99904114, rec: 0.99904114, f1: 0.99904114
Quality on test dataset: 
time to finish evaluation: 2.22s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 543	 0	 0.0	 0.0	 0.0
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	

...

Quality on test dataset: 
time to finish evaluation: 2.16s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 2833	 0	 0.0	 0.0	 0.0
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 3059	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 16535	 0	 0.0	 0.0	 0.0
Neg	 0	 777	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 19422	 0	 0.0	 0.0	 0.0
Obj	 0	 8223	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 367	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
Trigger	 0	 46	 0	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51262 fn: 8570 labels: 16
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 275/300 started, lr: 0.0012658229, dataset size: 22064
Epoch 275/300 - 13.49s - loss: 3.057042 - batches: 691
Quality on validation dataset (20.0%), validation examples = 4412
time to finish evaluation: 0.94s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 1224	 0	 0	 1.0	 1.0	

Epoch 281/300 - 13.41s - loss: 3.7222273 - batches: 691
Quality on validation dataset (20.0%), validation examples = 4412
time to finish evaluation: 0.94s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 1224	 0	 0	 1.0	 1.0	 1.0
Loc	 1869	 0	 0	 1.0	 1.0	 1.0
Act	 3510	 0	 0	 1.0	 1.0	 1.0
Neg	 280	 0	 0	 1.0	 1.0	 1.0
0	 5431	 0	 0	 1.0	 1.0	 1.0
Obj	 1567	 0	 0	 1.0	 1.0	 1.0
Pos	 295	 0	 0	 1.0	 1.0	 1.0
Trigger	 425	 0	 0	 1.0	 1.0	 1.0
tp: 14601 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Quality on test dataset: 
time to finish evaluation: 2.15s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 2234	 0	 0.0	 0.0	 0.0
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 2377	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 16477	 0	 0.0	 0.0	 0.0
Neg	 0	 646	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 20070	 0	 0.0	 0.0	 0.0
Obj	 0	 9084	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 

Quality on test dataset: 
time to finish evaluation: 2.14s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 2385	 0	 0.0	 0.0	 0.0
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 2425	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 17511	 0	 0.0	 0.0	 0.0
Neg	 0	 788	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 20897	 0	 0.0	 0.0	 0.0
Obj	 0	 7011	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 223	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
Trigger	 0	 20	 0	 0.0	 0.0	 0.0
B-PER	 0	 0	 1827	 0.0	 0.0	 0.0
tp: 0 fp: 51260 fn: 8570 labels: 16
Macro-average	 prec: 0.0, rec: 0.0, f1: NaN
Micro-average	 prec: 0.0, rec: 0.0, f1: 0.0
Epoch 288/300 started, lr: 0.0012320329, dataset size: 22064
Epoch 288/300 - 13.54s - loss: 3.727872 - batches: 691
Quality on validation dataset (20.0%), validation examples = 4412
time to finish evaluation: 0.93s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 1224	 0	 0	 1.0	 1.0	

Epoch 294/300 - 13.52s - loss: 2.5319324 - batches: 691
Quality on validation dataset (20.0%), validation examples = 4412
time to finish evaluation: 0.94s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 1224	 0	 0	 1.0	 1.0	 1.0
Loc	 1869	 0	 0	 1.0	 1.0	 1.0
Act	 3510	 0	 0	 1.0	 1.0	 1.0
Neg	 280	 0	 0	 1.0	 1.0	 1.0
0	 5431	 0	 0	 1.0	 1.0	 1.0
Obj	 1567	 0	 0	 1.0	 1.0	 1.0
Pos	 295	 0	 0	 1.0	 1.0	 1.0
Trigger	 425	 0	 0	 1.0	 1.0	 1.0
tp: 14601 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Quality on test dataset: 
time to finish evaluation: 2.15s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 2196	 0	 0.0	 0.0	 0.0
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 2465	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 15892	 0	 0.0	 0.0	 0.0
Neg	 0	 860	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 20927	 0	 0.0	 0.0	 0.0
Obj	 0	 8590	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 

time to finish evaluation: 0.93s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 1224	 0	 0	 1.0	 1.0	 1.0
Loc	 1869	 0	 0	 1.0	 1.0	 1.0
Act	 3510	 0	 0	 1.0	 1.0	 1.0
Neg	 280	 0	 0	 1.0	 1.0	 1.0
0	 5431	 0	 0	 1.0	 1.0	 1.0
Obj	 1567	 0	 0	 1.0	 1.0	 1.0
Pos	 295	 0	 0	 1.0	 1.0	 1.0
Trigger	 425	 0	 0	 1.0	 1.0	 1.0
tp: 14601 fp: 0 fn: 0 labels: 8
Macro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Micro-average	 prec: 1.0, rec: 1.0, f1: 1.0
Quality on test dataset: 
time to finish evaluation: 2.16s
label	 tp	 fp	 fn	 prec	 rec	 f1
Rqt	 0	 2216	 0	 0.0	 0.0	 0.0
B-LOC	 0	 0	 1837	 0.0	 0.0	 0.0
I-ORG	 0	 0	 751	 0.0	 0.0	 0.0
Loc	 0	 3011	 0	 0.0	 0.0	 0.0
I-MISC	 0	 0	 346	 0.0	 0.0	 0.0
Act	 0	 16400	 0	 0.0	 0.0	 0.0
Neg	 0	 741	 0	 0.0	 0.0	 0.0
I-LOC	 0	 0	 257	 0.0	 0.0	 0.0
0	 0	 22066	 0	 0.0	 0.0	 0.0
Obj	 0	 6537	 0	 0.0	 0.0	 0.0
I-PER	 0	 0	 1289	 0.0	 0.0	 0.0
Pos	 0	 264	 0	 0.0	 0.0	 0.0
B-MISC	 0	 0	 922	 0.0	 0.0	 0.0
B-ORG	 0	 0	 1341	 0.0	 0.0	 0.0
Trigger	 0	 27	 0	 0.0	 0.0	 0.

In [60]:
ner_model.stages[1].write().overwrite().save('NER_bert_20200221')

2023-01-17 11:55:00.582869: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [61]:
# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

bert = BertEmbeddings.pretrained('bert_base_cased', 'en')\
    .setInputCols(["sentence",'token'])\
    .setOutputCol("bert")\
    .setCaseSensitive(False)

loaded_ner_model = NerDLModel.load("NER_bert_20200221")\
   .setInputCols(["sentence", "token", "bert"])\
   .setOutputCol("ner")


ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_span')

custom_ner_pipeline = Pipeline(stages=[
      document_assembler, 
      sentence,
      tokenizer,
      bert,
      loaded_ner_model,
      ner_converter
 ])

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


2023-01-17 11:55:04.169877: W external/org_tensorflow/tensorflow/core/common_runtime/colocation_graph.cc:1218] Failed to place the graph without changing the devices of some resources. Some of the operations (that had to be colocated with resource generating operations) are not supported on the resources' devices. Current candidate devices are [
  /job:localhost/replica:0/task:0/device:CPU:0].
See below for details of this colocation group:
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=-1 requested_device_name_='/device:GPU:0' assigned_device_name_='' resource_device_name_='/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
AssignSub: CPU 
AddV2: CPU 
ScatterAdd: CPU 
NoOp: CPU 
Sqrt: CPU 
UnsortedSegmentSum: CPU 
RealDiv: CPU 
Const: CPU 
StridedSlice: CPU 
RandomUniform: CPU 
Unique: CPU 
Mul: CPU 
Add: CPU 
VariableV2: CPU 
Assign: CPU 
Identity: CPU 
Shape: CPU 
Cast: CPU 
Sub: CPU 
Gat

In [63]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = custom_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

{'document': ['navigate to kitchen'], 'ner_span': ['navigate to kitchen'], 'token': ['navigate', 'to', 'kitchen'], 'ner': ['Act', '0', 'Loc'], 'bert': ['navigate', 'to', 'kitchen'], 'sentence': ['navigate to kitchen']}
['Act', '0', 'Loc']


In [64]:
light.annotate("Navigate to kitchen")

{'document': ['Navigate to kitchen'],
 'ner_span': ['Navigate to kitchen'],
 'token': ['Navigate', 'to', 'kitchen'],
 'ner': ['Act', '0', 'Loc'],
 'bert': ['navigate', 'to', 'kitchen'],
 'sentence': ['Navigate to kitchen']}

In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

Enter Testing Text
kiss Gwen
['Act', 'Act']
Enter New Text
smoke weed
['Act', 'Obj']
Enter New Text
open cabinets
['Act', 'Obj']
Enter New Text
where am I
['Rqt', 'Loc', '0']
Enter New Text
where am i
['Rqt', 'Loc', '0']
Enter New Text
Where am I 
['Rqt', 'Loc', '0']
Enter New Text
what can I do here 
['Rqt', 'Rqt', '0', 'Act', 'Loc']
Enter New Text
what is there to do at this location
['Rqt', 'Rqt', '0', '0', 'Act', 'Rqt', '0', 'Loc']
